In [1]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

In [27]:
# URL cible
url = "https://www.agenceecofin.com/a-la-une/recherche-article"

In [28]:
# Fonction pour scraper une seule page
def scrape_page(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors de la récupération de la page : {url}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    articles = []

    # Sélection des éléments correspondant aux articles
    article_blocks = soup.find_all("td", class_="tsw")
    for block in article_blocks:
        try:
            # Récupérer le titre et le lien
            title_tag = block.find("a")
            title = title_tag.get_text(strip=True)
            link = title_tag["href"]

            # Construire l'URL absolue de l'article si nécessaire
            article_url = f"https://www.agenceecofin.com{link}"  # Modifier selon la structure réelle

            # Récupérer la source et la date
            source_tag = block.find("span", class_="news-source")
            author = source_tag.get_text(strip=True) if source_tag else "Auteur inconnu"
            date_tag = block.find("span", class_="f.nsa")
            date = date_tag.get_text(strip=True) if date_tag else "Date inconnue"

            # Récupérer le texte de l'article (dans la page principale)
            text_tag = block.find("div", class_="st")
            preview_text = text_tag.get_text(strip=True) if text_tag else "Texte non disponible"

            # Suivre le lien pour extraire le contenu complet de l'article
            full_text = scrape_article_content(article_url)

            # Ajouter l'article au tableau
            articles.append({
                "titre": title,
                "auteur": author,
                "date": date,
                "texte_previsualisation": preview_text,
                "texte_complet": full_text,
                "lien": article_url
            })
        except Exception as e:
            print(f"Erreur lors de l'extraction des données : {e}")
            continue

    return articles

# Fonction pour scraper le contenu complet d'un article
def scrape_article_content(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors de la récupération du contenu de l'article : {url}")
        return "Contenu inaccessible"

    soup = BeautifulSoup(response.content, "html.parser")

    # Sélectionner les paragraphes du contenu de l'article
    content_div = soup.find("div", class_="itemIntroText")
    if not content_div:
        return "Contenu non trouvé"

    paragraphs = content_div.find_all("p") #class_="texte textarticle"
    full_text = "\n".join([para.get_text(strip=True) for para in paragraphs])

    return full_text

# Fonction pour scraper toutes les pages
def scrape_all_page(url, max_pages=55):
    all_articles = []
    for page in range(1, max_pages + 1):
        print(f"Scraping la page {page}...")
        urls = f"{url}?page={page}"  # Modifier selon la structure réelle de la pagination
        articles = scrape_page(url)
        if not articles:  # Arrêter si aucune donnée n'est trouvée
            print("Aucune donnée trouvée, arrêt.")
            break
        all_articles.extend(articles)
    return all_articles

In [29]:
# Fonction pour sauvegarder les données dans un fichier JSON
def save_to_json(data, filename="articles.json"):
    try:
        with open(filename, "w", encoding="utf-8") as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
        print(f"Données sauvegardées dans le fichier {filename}")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde des données dans le fichier JSON : {e}")

In [ ]:
# Exemple d'utilisation
base_url = "https://www.agenceecofin.com/a-la-une/recherche-article"  # Remplacez par l'URL de la page principale
articless = scrape_all_page(base_url)

In [32]:
# Sauvegarder les articles dans un fichier JSON
save_to_json(articless, "articles_2.json")

Données sauvegardées dans le fichier articles_2.json
